In [ ]:
!pip install --quiet transformers datasets biopython pandas scikit-learn tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

dir_name = "cs598-evals-auto-af2/esm2_finetuning"
workdir = f"/content/drive/MyDrive/{dir_name}"
%cd $workdir

Mounted at /content/drive
/content/drive/MyDrive/cs598-evals-auto-af2/esm2_finetuning


In [ ]:
fasta_file = "PlasticDB.fasta"

from Bio import SeqIO

def load_fasta(fasta_file):
    """Load sequences from a FASTA file."""
    sequences = []

    with open(fasta_file, 'r') as f:
        for record in SeqIO.parse(f, "fasta"):
            sequences.append({
                'header': record.id,
                'sequence': str(record.seq)
            })

    return sequences

# Load sequences
sequences = load_fasta(fasta_file)
print(f"Loaded {len(sequences)} sequences")

Loaded 228 sequences


In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd

# Split data
train_seqs, val_seqs = train_test_split(sequences, test_size=0.15, random_state=42)

# Convert to HuggingFace datasets
train_df = pd.DataFrame(train_seqs)
val_df = pd.DataFrame(val_seqs)
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [ ]:
from transformers import EsmTokenizer, EsmForMaskedLM

model_name = "facebook/esm2_t12_35M_UR50D"

tokenizer = EsmTokenizer.from_pretrained(model_name)
model = EsmForMaskedLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/778 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/136M [00:00<?, ?B/s]

In [ ]:
def tokenize_sequences(examples, max_length=1024):
    return tokenizer(
        examples['sequence'],
        truncation=True,
        max_length=max_length,
        padding='max_length',
        return_special_tokens_mask=True
    )

train_tokenized = train_dataset.map(
    lambda examples: tokenize_sequences(examples),
    batched=True
)
val_tokenized = val_dataset.map(
    lambda examples: tokenize_sequences(examples),
    batched=True
)

Map:   0%|          | 0/193 [00:00<?, ? examples/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments

output_dir = "esm2_finetuned_plastic_degrading"
batch_size = 4

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True
)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=data_collator,
)

In [ ]:
!mkdir $output_dir

mkdir: cannot create directory ‘esm2_finetuned_plastic_degrading’: File exists


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=data_collator,
)

print("Starting fine-tuning...")
trainer.train()

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Starting fine-tuning...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: akash-arunabharathi (akash-arunabharathi-university-of-illinois-urbana-champaign) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


('esm2_finetuned_plastic_degrading/tokenizer_config.json',
 'esm2_finetuned_plastic_degrading/special_tokens_map.json',
 'esm2_finetuned_plastic_degrading/vocab.txt',
 'esm2_finetuned_plastic_degrading/added_tokens.json')

In [ ]:
import torch

def generate_proteins_via_masking(model, tokenizer, num_samples=5, template_length=200):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    model.eval()

    generated_sequences = []

    for i in range(num_samples):
        print(f"Generating sequence {i+1}/{num_samples}")
        input_text = tokenizer.cls_token + " " + tokenizer.mask_token * template_length

        inputs = tokenizer(input_text, return_tensors="pt").to(device)

        mask_positions = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

        for pos in mask_positions:
            with torch.no_grad():
                outputs = model(**inputs)

            logits = outputs.logits[0, pos, :]
            predicted_token_id = logits.argmax(-1).item()

            inputs.input_ids[0, pos] = predicted_token_id

        generated_seq = tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True)
        generated_sequences.append(generated_seq)

    return generated_sequences

generated_proteins = generate_proteins_via_masking(
    model,
    tokenizer,
    num_samples=5,
    template_length=200
)

with open(f"{output_dir}/generated_proteins.fasta", "w") as f:
    for i, seq in enumerate(generated_proteins):
        f.write(f">generated_plastic_degrading_protein_{i+1}\n")
        f.write(f"{seq}\n")

from google.colab import files
files.download(f"{output_dir}/generated_proteins.fasta")

Generating sequence 1/5
Generating sequence 2/5
Generating sequence 3/5
Generating sequence 4/5
Generating sequence 5/5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch

def generate_proteins_via_infilling(model, tokenizer, validation_sequences, num_samples=5,
                                   mask_ratio=0.3, iterations=10):
    """Generate proteins by iteratively infilling masked positions in validation sequences"""
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    model.eval()

    generated_sequences = []

    import random
    seed_sequences = []
    for _ in range(num_samples):
        rand_idx = random.randint(0, len(validation_sequences) - 1)
        seed_sequences.append(validation_sequences[rand_idx]['sequence'])

    for i, original_seq in enumerate(seed_sequences):
        print(f"Generating sequence {i+1}/{num_samples}")

        current_seq = original_seq

        for iter in range(iterations):
            print(f"  Iteration {iter+1}/{iterations}")

            seq_list = list(current_seq)

            num_to_mask = int(len(seq_list) * mask_ratio)
            mask_positions = random.sample(range(len(seq_list)), num_to_mask)

            for pos in mask_positions:
                seq_list[pos] = tokenizer.mask_token

            masked_seq = ''.join(seq_list)

            input_text = tokenizer.cls_token + " " + masked_seq
            inputs = tokenizer(input_text, return_tensors="pt").to(device)

            mask_positions = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

            with torch.no_grad():
                outputs = model(**inputs)

            for pos in mask_positions:
                logits = outputs.logits[0, pos, :]

                valid_aa_indices = []
                for aa in "ACDEFGHIKLMNPQRSTVWY":
                    aa_token_id = tokenizer.convert_tokens_to_ids(aa)
                    if aa_token_id != tokenizer.unk_token_id:
                        valid_aa_indices.append(aa_token_id)

                valid_logits = logits[valid_aa_indices]
                probs = torch.nn.functional.softmax(valid_logits, dim=0)

                next_token_idx = torch.multinomial(probs, 1).item()
                next_aa_id = valid_aa_indices[next_token_idx]
                next_aa = tokenizer.convert_ids_to_tokens(next_aa_id)

                inputs.input_ids[0, pos] = next_aa_id

            current_seq = tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True)

        generated_sequences.append(current_seq)

    return generated_sequences

validation_sequences = val_seqs

generated_proteins = generate_proteins_via_infilling(
    model,
    tokenizer,
    validation_sequences,
    num_samples=5,
    mask_ratio=0.3,
    iterations=10
)

with open(f"{output_dir}/generated_proteins_infilling.fasta", "w") as f:
    for i, seq in enumerate(generated_proteins):
        f.write(f">generated_plastic_degrading_protein_{i+1}\n")
        f.write(f"{seq}\n")

from google.colab import files
files.download(f"{output_dir}/generated_proteins_infilling.fasta")

Generating sequence 1/5
  Iteration 1/10
  Iteration 2/10
  Iteration 3/10
  Iteration 4/10
  Iteration 5/10
  Iteration 6/10
  Iteration 7/10
  Iteration 8/10
  Iteration 9/10
  Iteration 10/10
Generating sequence 2/5
  Iteration 1/10
  Iteration 2/10
  Iteration 3/10
  Iteration 4/10
  Iteration 5/10
  Iteration 6/10
  Iteration 7/10
  Iteration 8/10
  Iteration 9/10
  Iteration 10/10
Generating sequence 3/5
  Iteration 1/10
  Iteration 2/10
  Iteration 3/10
  Iteration 4/10
  Iteration 5/10
  Iteration 6/10
  Iteration 7/10
  Iteration 8/10
  Iteration 9/10
  Iteration 10/10
Generating sequence 4/5
  Iteration 1/10
  Iteration 2/10
  Iteration 3/10
  Iteration 4/10
  Iteration 5/10
  Iteration 6/10
  Iteration 7/10
  Iteration 8/10
  Iteration 9/10
  Iteration 10/10
Generating sequence 5/5
  Iteration 1/10
  Iteration 2/10
  Iteration 3/10
  Iteration 4/10
  Iteration 5/10
  Iteration 6/10
  Iteration 7/10
  Iteration 8/10
  Iteration 9/10
  Iteration 10/10


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def generate_proteins_via_infilling(model, tokenizer, validation_sequences, num_samples=5,
                                  mask_ratio=0.3, iterations=5, target_length=400):
    """Generate proteins by iteratively infilling masked positions in validation sequences"""
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    model.eval()

    generated_sequences = []

    import random
    seed_sequences = []
    for _ in range(num_samples):
        rand_idx = random.randint(0, len(validation_sequences) - 1)
        orig_seq = validation_sequences[rand_idx]['sequence']

        if len(orig_seq) > target_length:
            start_idx = random.randint(0, len(orig_seq) - target_length)
            trimmed_seq = orig_seq[start_idx:start_idx + target_length]
        else:
            trimmed_seq = orig_seq

        seed_sequences.append(trimmed_seq)

    for i, original_seq in enumerate(seed_sequences):
        print(f"Generating sequence {i+1}/{num_samples}")
        print(f"  Starting with sequence of length {len(original_seq)}")


        current_seq = original_seq

        for iter in range(iterations):
            print(f"  Iteration {iter+1}/{iterations}")

            seq_list = list(current_seq)

            num_to_mask = int(len(seq_list) * mask_ratio)
            num_to_mask = min(num_to_mask, 50)

            mask_positions = random.sample(range(len(seq_list)), num_to_mask)

            for pos in mask_positions:
                seq_list[pos] = tokenizer.mask_token

            masked_seq = ''.join(seq_list)

            max_model_length = 1022

            if len(masked_seq) > max_model_length:
                chunk_results = []
                chunk_size = max_model_length
                overlap = 50

                for start_idx in range(0, len(masked_seq), chunk_size - overlap):
                    end_idx = min(start_idx + chunk_size, len(masked_seq))
                    chunk = masked_seq[start_idx:end_idx]

                    input_text = tokenizer.cls_token + " " + chunk
                    inputs = tokenizer(input_text, return_tensors="pt").to(device)

                    chunk_mask_positions = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

                    if len(chunk_mask_positions) == 0:
                        chunk_results.append(chunk)
                        continue

                    with torch.no_grad():
                        outputs = model(**inputs)

                    for pos in chunk_mask_positions:
                        logits = outputs.logits[0, pos, :]

                        valid_aa_indices = []
                        for aa in "ACDEFGHIKLMNPQRSTVWY":
                            aa_token_id = tokenizer.convert_tokens_to_ids(aa)
                            if aa_token_id != tokenizer.unk_token_id:
                                valid_aa_indices.append(aa_token_id)

                        valid_logits = logits[valid_aa_indices]
                        probs = torch.nn.functional.softmax(valid_logits, dim=0)
                        next_token_idx = torch.multinomial(probs, 1).item()
                        next_aa_id = valid_aa_indices[next_token_idx]

                        inputs.input_ids[0, pos] = next_aa_id

                    processed_chunk = tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True)
                    chunk_results.append(processed_chunk)

                if len(chunk_results) == 1:
                    current_seq = chunk_results[0]
                else:
                    merged_seq = chunk_results[0][:-overlap]
                    for i in range(1, len(chunk_results) - 1):
                        merged_seq += chunk_results[i][overlap:-overlap]
                    merged_seq += chunk_results[-1][overlap:]
                    current_seq = merged_seq
            else:
                input_text = tokenizer.cls_token + " " + masked_seq
                inputs = tokenizer(input_text, return_tensors="pt").to(device)

                mask_positions = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

                if len(mask_positions) > 0:
                    with torch.no_grad():
                        outputs = model(**inputs)

                    for pos in mask_positions:
                        logits = outputs.logits[0, pos, :]

                        valid_aa_indices = []
                        for aa in "ACDEFGHIKLMNPQRSTVWY":
                            aa_token_id = tokenizer.convert_tokens_to_ids(aa)
                            if aa_token_id != tokenizer.unk_token_id:
                                valid_aa_indices.append(aa_token_id)

                        valid_logits = logits[valid_aa_indices]
                        probs = torch.nn.functional.softmax(valid_logits, dim=0)
                        next_token_idx = torch.multinomial(probs, 1).item()
                        next_aa_id = valid_aa_indices[next_token_idx]

                        inputs.input_ids[0, pos] = next_aa_id

                    current_seq = tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True)

        print(f"  Final sequence length: {len(current_seq)}")
        generated_sequences.append(current_seq)

    return generated_sequences

generated_proteins = generate_proteins_via_infilling(
    model,
    tokenizer,
    validation_sequences=val_seqs,
    num_samples=5,
    mask_ratio=0.3,
    iterations=5,
    target_length=400
)

with open(f"{output_dir}/generated_proteins_infilling.fasta", "w") as f:
    for i, seq in enumerate(generated_proteins):
        f.write(f">generated_plastic_degrading_protein_{i+1}\n")
        f.write(f"{seq}\n")

from google.colab import files
files.download(f"{output_dir}/generated_proteins_infilling.fasta")

Generating sequence 1/5
  Starting with sequence of length 400
  Iteration 1/5
  Iteration 2/5
  Iteration 3/5
  Iteration 4/5
  Iteration 5/5
  Final sequence length: 821
Generating sequence 2/5
  Starting with sequence of length 240
  Iteration 1/5
  Iteration 2/5
  Iteration 3/5
  Iteration 4/5
  Iteration 5/5
  Final sequence length: 681
Generating sequence 3/5
  Starting with sequence of length 301
  Iteration 1/5
  Iteration 2/5
  Iteration 3/5
  Iteration 4/5
  Iteration 5/5
  Final sequence length: 813
Generating sequence 4/5
  Starting with sequence of length 400
  Iteration 1/5
  Iteration 2/5
  Iteration 3/5
  Iteration 4/5
  Iteration 5/5
  Final sequence length: 823
Generating sequence 5/5
  Starting with sequence of length 305
  Iteration 1/5
  Iteration 2/5
  Iteration 3/5
  Iteration 4/5
  Iteration 5/5
  Final sequence length: 811


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>